# Classifier 

## Fiction (with examples) 

In [3]:
 import pandas as pd

In [1]:
# set directory to get .env file
%cd "/Users/stevenmesquiti/Desktop/working-with-lyle/Text-Annotation/blind-round-2" 

/Users/stevenmesquiti/Desktop/working-with-lyle/Text-Annotation/blind-round-2


In [6]:
df=pd.read_csv('/Users/stevenmesquiti/Dropbox/Collected Texts/Atlantic-csv/500-words-Atlantic-CSV-Formal-Informal-Fiction-Full.csv')
df.head(10) #grab the text column 

,Unnamed: 0,gpt_formality_ratings,Text,file_name,year,month,gpt_coding_raw,gpt_coding_final
0,2476,1.0,The Story of the Queen The Story of the Queen ...,637338-the-story-of-the-queen.txt,1903,11,Fiction.,fiction
1,6773,0.0,Burn Your Maps Burn Your Maps Robyn Joy Leff A...,302385-burn-your-maps.txt,2002,1,fiction,fiction
2,6116,0.0,The Neurotic S Notebook The Neurotic S Noteboo...,658882-the-neurotic-s-notebook.txt,1962,6,The text is fiction.,fiction
3,3047,1.0,Their Wedding Journey Their Wedding Journey Wi...,630459-their-wedding-journey.txt,1871,8,fiction.,fiction
4,4092,1.0,De Grey : A Romance De Grey : A Romance IT was...,629196-de-grey-a-romance.txt,1868,7,fiction,fiction
5,3218,0.0,PS PS Jill McCorkle What I know now is that I ...,307540-ps.txt,2009,8,fiction,fiction
6,3768,1.0,Once I Lived Once I Lived Karl Kirchwey A poem...,559160-once-i-lived.txt,2018,6,The text is fiction.,fiction
7,3463,0.0,Shanghai Murmur Shanghai Murmur Te-Ping Chen A...,617269-shanghai-murmur.txt,2021,1,fiction,fiction
8,3414,0.0,Parents Parents Greg Delanty What do any of us...,308024-parents.txt,2010,5,fiction,fiction
9,1192,0.0,Journal of a Plague Year Journal of a Plague Y...,664183-journal-of-a-plague-year.txt,1973,8,fiction,fiction


In [ ]:
num_rows = len(df)
print(num_rows)

In [ ]:
#Fiction examples 
examples=pd.read_csv('/Users/stevenmesquiti/Desktop/working-with-lyle/Text-Annotation/blind-round-2/GPT-examples/Atlantic-Fiction-Examples.csv', index_col=0)
examples.head()

In [ ]:
inputs = df.Text.values #save the text

In [ ]:
# View the first 5 text values
print(inputs[:5])

In [ ]:
#load other dependencies 
import argparse
import json
import os

import numpy as np
import openai
import pandas as pd
import random
import time
from tqdm.notebook import tqdm
from dotenv import load_dotenv
import os
import threading
import time

In [ ]:
title='Atlantic'
subtitle='Fiction'
stim_set=title+'-'+subtitle
seed=1 #set seed to make sure we get reproducable results 
temperature=0.1 #want a low baking temp to have little variability or creativity 
engine='gpt-3.5-turbo' #change this to use different models
n_context=1
cache = True   
resume=False
# MIDI='freq' #or 'name', 'number', 'freq'
audience='People'
item='Text'

load_dotenv()

#apiKey = os.environ.get('steven_key') #for stevens key 

apiKey = os.environ.get('angela_key') #for angelas key 

openai.api_key = apiKey 

if cache:
    random.seed(seed)

In [ ]:
def generate_prompt(current):#(example_idxs, current_pair):
    prompt = f"""Your task is to classify a piece of as either formal or informal. 
You will be provided with two examples of formal text and two examples of informal text. Answer only with a number: 1 if formal, and 0 if informal.  

Text: {examples.Example.iloc[0]} 
Rating: {examples.Label.iloc[0]}
    
Text: {examples.Example.iloc[1]}
Rating: {examples.Label.iloc[1]}

Text: {examples.Example.iloc[2]}
Rating: {examples.Label.iloc[2]}

Text: {examples.Example.iloc[3]}
Rating: {examples.Label.iloc[3]}
    
Text: {inputs[current]}
Rating:
"""
    if n_context==0:
        prompt = f"""Is the following piece of text formal or informal? Answer only with a number: 1 if formal, and 0 if informal Here is the text:\n{inputs[current]}"""
    
    return prompt

cache_folder = f'cache/{stim_set}'
os.makedirs(cache_folder, exist_ok=True)
def create_cache_filename():
    filename = f'{stim_set}-{n_context}-{engine}-{temperature}-{seed}'
    # if args.shuffle_context_each_draw:
    #     filename += '-shuffle'
    return os.path.join(cache_folder, filename + '.json')

if not resume:
    visited = []
    predicted_ratings = np.zeros((len(inputs)))
    request_count = 0

cache = {}

if cache and os.path.exists(create_cache_filename()):
    cache = json.load(open(create_cache_filename()))
cached_keys = list(cache.keys())

for idx1, bname1 in enumerate(tqdm(inputs)):
    current = idx1
    if current in visited and predicted_ratings[current]!=0:
        print(f'Already visited {current}')
        continue

    visited.append(current)

    key = f'{current}'
    if key in cached_keys:
        choices = cache[key]['choices']
        print('Using cached choices for key', key)
    else:
        prompt = generate_prompt(current)#(example_idxs, current_pair)
        response=False
        i=0
        while not response:
            i+=1
            try:
                response = openai.ChatCompletion.create(
                    model=engine,
                    messages=[{'role':'user', 'content':prompt}],
                    temperature=temperature,
                    timeout=10
                )
            except Exception as e:
                print(f'Attempt {i} failed')
                time.sleep(5)
        choices = [dict(choice.items()) for choice in response.choices]

        cache[key] = {
            'prompt': prompt,
            'choices': choices,
            'created': response.created
        }

        request_count += 1
        if cache and request_count % 5 == 0:
            json.dump(cache, open(create_cache_filename(), 'w'))

    try:
        answer = choices[0]['message']['content'].replace('\n', '').strip()
        predicted_ratings[idx1] = int(answer)
    except:
        print('Error', cache[key])

os.makedirs(f'predictions/{stim_set}', exist_ok=True)
np.save(f'predictions/{stim_set}/{stim_set}-{n_context}-{engine}-{temperature}-{seed}.npy', predicted_ratings)


In [ ]:
df=pd.read_csv("/Users/stevenmesquiti/Dropbox/Collected Texts/Atlantic-csv/600-words-Atlantic-CSV-Formal-Informal-Fiction-Full.csv")
# df_gpt4 = np.zeros((predicted_ratings.size, 3))
# df_gpt4[np.arange(predicted_ratings.size), predicted_ratings.astype(int)] = 1
predicted_ratings=np.load(f'predictions/{stim_set}/{stim_set}-{n_context}-{engine}-{temperature}-{seed}.npy', allow_pickle=True)
df.head()

In [ ]:
predicted_ratings # just to check it ran smoothly
df.columns

In [ ]:
fiction_df_gpt35_250=pd.DataFrame(predicted_ratings, columns=['gpt_formality_ratings'])
fiction_df_gpt35_250['Text']=df.Text.values
fiction_df_gpt35_250['file_name']=df.file_name.values

In [ ]:
#save the work
output_directory = '/Users/stevenmesquiti/Desktop/working-with-lyle/Text-Annotation/blind-round-2/output/final/'
os.makedirs(output_directory, exist_ok=True)

fiction_df_gpt35_250.to_csv(os.path.join(output_directory, f'{stim_set}-{n_context}-{engine}-{temperature}-{seed}.csv'), index=False)

In [ ]:
### need to work on this 

# emotions=np.array(['yes','no'])
df=df.reset_index()
# one_hot = pd.get_dummies(df['human'])
df=df.rename(columns={'human':'overall_inter_rater_reliability'})
# df=df.join(one_hot)
matches=np.all(df_gpt4[['offensive']]==df.reset_index(drop=True)[['offensive']],axis=1)
np.mean(matches)

## Non-fiction 

In [17]:
 import pandas as pd
    # set directory to get .env file

In [18]:
%cd "/Users/stevenmesquiti/Desktop/working-with-lyle/Text-Annotation/blind-round-2" 

/Users/stevenmesquiti/Desktop/working-with-lyle/Text-Annotation/blind-round-2


In [32]:
#read in the 5 separate dataframes

#1 - done
#df=pd.read_csv('/Users/stevenmesquiti/Dropbox/Collected Texts/Atlantic-csv/500-words-Atlantic-CSV-Formal-Informal-NonFiction-Full_1.csv')


#2 -done
#df=pd.read_csv('/Users/stevenmesquiti/Dropbox/Collected Texts/Atlantic-csv/500-words-Atlantic-CSV-Formal-Informal-NonFiction-Full_2.csv')

#3 - done
#df=pd.read_csv('/Users/stevenmesquiti/Dropbox/Collected Texts/Atlantic-csv/500-words-Atlantic-CSV-Formal-Informal-NonFiction-Full_3.csv')

#4 - done
#df=pd.read_csv('/Users/stevenmesquiti/Dropbox/Collected Texts/Atlantic-csv/500-words-Atlantic-CSV-Formal-Informal-NonFiction-Full_4.csv')

#5 - done
df=pd.read_csv('/Users/stevenmesquiti/Dropbox/Collected Texts/Atlantic-csv/500-words-Atlantic-CSV-Formal-Informal-NonFiction-Full_5.csv')


df.head() #grab the text column 


,Unnamed: 0,file_name,Text,year,month,gpt_coding_raw,gpt_coding_final,Tokens
0,35071,641130-the-wealth-we-wasted.txt,The Wealth We Wasted The Wealth We Wasted A. W...,1952,8,non-fiction,non-fiction,"['The', 'Wealth', 'We', 'Wasted', 'The', 'Weal..."
1,35072,641803-the-peripatetic-reviewer.txt,The Peripatetic Reviewer The Peripatetic Revie...,1952,8,Non-fiction.,non-fiction,"['The', 'Peripatetic', 'Reviewer', 'The', 'Per..."
2,35073,641018-a-life-of-disagreement.txt,A Life of Disagreement A Life of Disagreement ...,1952,8,Non-fiction.,non-fiction,"['A', 'Life', 'of', 'Disagreement', 'A', 'Life..."
3,35075,641902-spain.txt,Spain Spain on the World TodayDURING the riots...,1952,8,Non-fiction.,non-fiction,"['Spain', 'Spain', 'on', 'the', 'World', 'Toda..."
4,35076,642759-record-reviews.txt,Record Reviews Record Reviews John M. Conly by...,1952,8,Non-fiction.,non-fiction,"['Record', 'Reviews', 'Record', 'Reviews', 'Jo..."


In [33]:
num_rows = df.shape[0]

print("Number of rows:", num_rows)

Number of rows: 7342


In [34]:
import tiktoken
encoding = tiktoken.get_encoding("cl100k_base")
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
# Specify the maximum token count threshold
max_token_count = 3000


In [35]:
#dealing with too many tokens NEED TO RUN TO REMOVE SHITTY DATA

# Define a function to count tokens in a text
def count_tokens(text):
    tokens = encoding.encode(text)
    return len(tokens)

# Apply token count function to 'Text' column and filter based on maximum token count
df = df[df['Text'].apply(count_tokens) <= max_token_count]

# Print the filtered DataFrame
num_rows = df.shape[0]

print("Number of rows:", num_rows)

Number of rows: 7342


In [36]:
#Fiction examples 
examples=pd.read_csv('/Users/stevenmesquiti/Desktop/working-with-lyle/Text-Annotation/blind-round-2/GPT-examples/Atlantic-Nonfiction-Examples.csv')
examples.head()

,file_name,year,Example,Label,formality_rating,gpt_coding_final,angela_formality_rating,lyle_formality_rating,ben_formality_rating,steven_formality_rating,interrater_reliability,overall_inter_rater_reliability,GPT3_5_NO_example,no_example_cor,GPT3_5_with_example,example_cor,GPT3_5_2_example
0,628188-the-question-of-the-hour.txt,1861,"However true this may be in the main, — and it...",1,NaN,non-fiction,1,1,1,1,1.000000,0.280369,1,NaN,1,NaN,1
1,633747-why-our-science-students-go-to-germany.txt,1889,Doubtless each individual is influenced by a v...,1,NaN,non-fiction,1,1,1,1,1.000000,0.280369,1,NaN,1,NaN,1
2,659289-why-im-a-cop-interviews-from-a-reporter...,1969,New York policeman for ten years; recently bec...,0,NaN,non-fiction,0,0,0,0,1.000000,0.280369,0,NaN,0,NaN,0
3,307617-how-american-health-care-killed-my-fath...,2009,"Like every grieving family member, I looked fo...",0,NaN,non-fiction,1,0,0,0,-0.222222,0.280369,0,NaN,0,NaN,0


In [37]:
inputs = df.Text.values #save the text
print(inputs[:3])

["The Wealth We Wasted The Wealth We Wasted A. W. Smith A. W. SMITH , who was born in Spain of British parents , lived in Russia as a boy and was educated at Shrewsbury and at Sandhurst . He served with the British Army throughout both World Wars . For some years he has made his home in Ipswich , Massachusetts , where he wrote his books , The Captain Departed and The Sword and the Rose , and a number of short stories which have appeared in the Atlantic . He has recently become an American citizen , and is today a Vice President on the staff of The Conservation Foundation . by A. W. SMITH1IT is probably near-heresy to suggest that the pioneer forefathers may not have been well equipped to deal intelligently with the problems posed in their new environment . Many of their practices , so evil and so wasteful in the coldly superior light of the middle of the twentieth century , may have been forced on them by the demands of the moment . But these are not sufficient to account for the destr

In [38]:
#load other dependencies 
import argparse
import json
import os
import numpy as np
import openai
import pandas as pd
import random
import time
from tqdm.notebook import tqdm
from dotenv import load_dotenv

In [39]:
title='Atlantic'
subtitle='Non-fiction'
stim_set=title+'-'+subtitle
seed=1
temperature=0.1 #want a low baking temp to have little variability or creativity 
engine='gpt-3.5-turbo' #change this to use different models
n_context=1
cache = True   
resume=False
# MIDI='freq' #or 'name', 'number', 'freq'
audience='People'
item='Text'

load_dotenv()

apiKey = os.environ.get('angela_key') #for angelas key 

openai.api_key = apiKey 

if cache:
    random.seed(seed)

In [40]:
def generate_prompt(current):#(example_idxs, current_pair):
    prompt = f"""Your task is to classify a piece of as either formal or informal. 
You will be provided with two examples of formal text and two examples of informal text. Answer only with a number: 1 if formal, and 0 if informal.  

Text: {examples.Example.iloc[0]} 
Rating: {examples.Label.iloc[0]}
    
Text: {examples.Example.iloc[1]}
Rating: {examples.Label.iloc[1]}

Text: {examples.Example.iloc[2]}
Rating: {examples.Label.iloc[2]}

Text: {examples.Example.iloc[3]}
Rating: {examples.Label.iloc[3]}
    
Text: {inputs[current]}
Rating:
"""
    if n_context==0:
        prompt = f"""Is the following piece of text formal or informal? Answer only with a number: 1 if formal, and 0 if informal Here is the text:\n{inputs[current]}"""
    
    return prompt

cache_folder = f'cache/{stim_set}'
os.makedirs(cache_folder, exist_ok=True)

def create_cache_filename():
    filename = f'{stim_set}-{n_context}-{engine}-{temperature}-{seed}'
    # if args.shuffle_context_each_draw:
    #     filename += '-shuffle'
    return os.path.join(cache_folder, filename + '.json')

if not resume:
    visited = []
    predicted_ratings = np.zeros((len(inputs)))
    request_count = 0

cache = {}

if cache and os.path.exists(create_cache_filename()):
    cache = json.load(open(create_cache_filename()))
cached_keys = list(cache.keys())

for idx1, bname1 in enumerate(tqdm(inputs)):
    current = idx1
    if current in visited and predicted_ratings[current] != 0:
        print(f'Already visited {current}')
        continue

    visited.append(current)

    key = f'{current}'
    if key in cached_keys:
        choices = cache[key]['choices']
        print('Using cached choices for key', key)
    else:
        prompt = generate_prompt(current)
        response = False
        i = 0
        while not response:
            i += 1
            try:
                response = openai.ChatCompletion.create(
                    model=engine,
                    messages=[{'role': 'user', 'content': prompt}],
                    temperature=temperature,
                    timeout=10
                )
            except Exception as e:
                print(f'Attempt {i} failed\n{e}\n')
                
                time.sleep(7) #7 second delay 
        choices = [dict(choice.items()) for choice in response.choices]

        cache[key] = {
            'prompt': prompt,
            'choices': choices,
            'created': response.created
        }

        request_count += 1
        if cache and request_count % 5 == 0:
            json.dump(cache, open(create_cache_filename(), 'w'))

    try:
        answer = choices[0]['message']['content'].replace('\n', '').strip()
        predicted_ratings[idx1] = int(answer)
    except Exception as e:
        print(f'Error: {e}\n\n{cache[key]}')

os.makedirs(f'predictions/{stim_set}', exist_ok=True)
np.save(f'predictions/{stim_set}/{stim_set}-{n_context}-{engine}-{temperature}-{seed}.npy', predicted_ratings)


  0%|          | 0/7342 [00:00<?, ?it/s]

Attempt 1 failed
Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600)

Attempt 1 failed
Bad gateway. {"error":{"code":502,"message":"Bad gateway.","param":null,"type":"cf_bad_gateway"}} 502 {'error': {'code': 502, 'message': 'Bad gateway.', 'param': None, 'type': 'cf_bad_gateway'}} {'Date': 'Fri, 30 Jun 2023 14:25:17 GMT', 'Content-Type': 'application/json', 'Content-Length': '84', 'Connection': 'keep-alive', 'X-Frame-Options': 'SAMEORIGIN', 'Referrer-Policy': 'same-origin', 'Cache-Control': 'private, max-age=0, no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': 'Thu, 01 Jan 1970 00:00:01 GMT', 'Server': 'cloudflare', 'CF-RAY': '7df70c49be3c43f9-EWR', 'alt-svc': 'h3=":443"; ma=86400'}

Attempt 1 failed
The server is overloaded or not ready yet.

Attempt 1 failed
The server is overloaded or not ready yet.

Attempt 1 failed
Rate limit reached for default-gpt-3.5-turbo in organization org-D9XDwg2Sw2y54gEdPpW9M

Attempt 1 failed
Rate limit reached for default-gpt-3.5-turbo in organization org-D9XDwg2Sw2y54gEdPpW9Mqvj on tokens per min. Limit: 90000 / min. Current: 88749 / min. Contact us through our help center at help.openai.com if you continue to have issues.

Attempt 1 failed
Rate limit reached for default-gpt-3.5-turbo in organization org-D9XDwg2Sw2y54gEdPpW9Mqvj on tokens per min. Limit: 90000 / min. Current: 88904 / min. Contact us through our help center at help.openai.com if you continue to have issues.

Attempt 1 failed
Bad gateway. {"error":{"code":502,"message":"Bad gateway.","param":null,"type":"cf_bad_gateway"}} 502 {'error': {'code': 502, 'message': 'Bad gateway.', 'param': None, 'type': 'cf_bad_gateway'}} {'Date': 'Fri, 30 Jun 2023 15:39:52 GMT', 'Content-Type': 'application/json', 'Content-Length': '84', 'Connection': 'keep-alive', 'X-Frame-Options': 'SAMEORIGIN', 'Referrer-Policy': 'same-origin', 'Cache-Control': 'private, max-age=0, no-store, no-cache, must-revalidate, post-c

Attempt 1 failed
Rate limit reached for default-gpt-3.5-turbo in organization org-D9XDwg2Sw2y54gEdPpW9Mqvj on tokens per min. Limit: 90000 / min. Current: 89379 / min. Contact us through our help center at help.openai.com if you continue to have issues.

Attempt 1 failed
Rate limit reached for default-gpt-3.5-turbo in organization org-D9XDwg2Sw2y54gEdPpW9Mqvj on tokens per min. Limit: 90000 / min. Current: 89296 / min. Contact us through our help center at help.openai.com if you continue to have issues.

Attempt 1 failed
Rate limit reached for default-gpt-3.5-turbo in organization org-D9XDwg2Sw2y54gEdPpW9Mqvj on tokens per min. Limit: 90000 / min. Current: 88956 / min. Contact us through our help center at help.openai.com if you continue to have issues.

Attempt 1 failed
Rate limit reached for default-gpt-3.5-turbo in organization org-D9XDwg2Sw2y54gEdPpW9Mqvj on tokens per min. Limit: 90000 / min. Current: 88925 / min. Contact us through our help center at help.openai.com if you contin

Attempt 1 failed
Bad gateway. {"error":{"code":502,"message":"Bad gateway.","param":null,"type":"cf_bad_gateway"}} 502 {'error': {'code': 502, 'message': 'Bad gateway.', 'param': None, 'type': 'cf_bad_gateway'}} {'Date': 'Fri, 30 Jun 2023 18:20:07 GMT', 'Content-Type': 'application/json', 'Content-Length': '84', 'Connection': 'keep-alive', 'X-Frame-Options': 'SAMEORIGIN', 'Referrer-Policy': 'same-origin', 'Cache-Control': 'private, max-age=0, no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': 'Thu, 01 Jan 1970 00:00:01 GMT', 'Server': 'cloudflare', 'CF-RAY': '7df8644b980ec468-EWR', 'alt-svc': 'h3=":443"; ma=86400'}

Attempt 1 failed
Bad gateway. {"error":{"code":502,"message":"Bad gateway.","param":null,"type":"cf_bad_gateway"}} 502 {'error': {'code': 502, 'message': 'Bad gateway.', 'param': None, 'type': 'cf_bad_gateway'}} {'Date': 'Fri, 30 Jun 2023 18:25:45 GMT', 'Content-Type': 'application/json', 'Content-Length': '84', 'Connection': 'keep-alive', 'X-Frame-

Attempt 1 failed
Rate limit reached for default-gpt-3.5-turbo in organization org-D9XDwg2Sw2y54gEdPpW9Mqvj on tokens per min. Limit: 90000 / min. Current: 88880 / min. Contact us through our help center at help.openai.com if you continue to have issues.

Attempt 2 failed
The server is overloaded or not ready yet.

Attempt 1 failed
Rate limit reached for default-gpt-3.5-turbo in organization org-D9XDwg2Sw2y54gEdPpW9Mqvj on tokens per min. Limit: 90000 / min. Current: 88867 / min. Contact us through our help center at help.openai.com if you continue to have issues.

Attempt 1 failed
Rate limit reached for default-gpt-3.5-turbo in organization org-D9XDwg2Sw2y54gEdPpW9Mqvj on tokens per min. Limit: 90000 / min. Current: 89190 / min. Contact us through our help center at help.openai.com if you continue to have issues.

Attempt 1 failed
Rate limit reached for default-gpt-3.5-turbo in organization org-D9XDwg2Sw2y54gEdPpW9Mqvj on tokens per min. Limit: 90000 / min. Current: 89095 / min. Contac

In [ ]:
#read in the 5 separate dataframes

#1 - done
#df=pd.read_csv('/Users/stevenmesquiti/Dropbox/Collected Texts/Atlantic-csv/500-words-Atlantic-CSV-Formal-Informal-NonFiction-Full_1.csv')


#2 - done
#df=pd.read_csv('/Users/stevenmesquiti/Dropbox/Collected Texts/Atlantic-csv/500-words-Atlantic-CSV-Formal-Informal-NonFiction-Full_2.csv')

#3
#df=pd.read_csv('/Users/stevenmesquiti/Dropbox/Collected Texts/Atlantic-csv/500-words-Atlantic-CSV-Formal-Informal-NonFiction-Full_3.csv')

#4
#df=pd.read_csv('/Users/stevenmesquiti/Dropbox/Collected Texts/Atlantic-csv/500-words-Atlantic-CSV-Formal-Informal-NonFiction-Full_4.csv')

#5
#df=pd.read_csv('/Users/stevenmesquiti/Dropbox/Collected Texts/Atlantic-csv/500-words-Atlantic-CSV-Formal-Informal-NonFiction-Full_5.csv')


df.head() #grab the text column # df_gpt4 = np.zeros((predicted_ratings.size, 3))
# df_gpt4[np.arange(predicted_ratings.size), predicted_ratings.astype(int)] = 1
predicted_ratings=np.load(f'predictions/{stim_set}/{stim_set}-{n_context}-{engine}-{temperature}-{seed}.npy', allow_pickle=True)
df.head()

In [41]:
#dealing with too many tokens NEED TO RUN TO REMOVE SHITTY DATA

# Define a function to count tokens in a text
def count_tokens(text):
    tokens = encoding.encode(text)
    return len(tokens)

# Apply token count function to 'Text' column and filter based on maximum token count
df = df[df['Text'].apply(count_tokens) <= max_token_count]

# Print the filtered DataFrame
num_rows = df.shape[0]

print("Number of rows:", num_rows)

Number of rows: 7342


In [42]:
predicted_ratings # just to check it ran smoothly
df.columns

Index(['Unnamed: 0', 'file_name', 'Text', 'year', 'month', 'gpt_coding_raw',
       'gpt_coding_final', 'Tokens'],
      dtype='object')

In [43]:
nonfiction_df_gpt35=pd.DataFrame(predicted_ratings, columns=['formality_ratings'])
nonfiction_df_gpt35['Text']=df.Text.values
nonfiction_df_gpt35['file_name']=df.file_name.values

In [44]:
#save the work
output_directory = '/Users/stevenmesquiti/Desktop/working-with-lyle/Text-Annotation/blind-round-2/output/final'
os.makedirs(output_directory, exist_ok=True)
nonfiction_df_gpt35.to_csv(os.path.join(output_directory, f'{stim_set}-{n_context}-{engine}-{temperature}-{seed}.csv'), index=False)